# NDWI Analysis - NL dams

( Description )

## Import libraries

In [1]:
pip install pymannkendall


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip3.10 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandarallel


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip3.10 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
import planetary_computer as pc
import geopandas as gpd
import h3
import pandas as pd

import rasterio
from rasterio import windows
from rasterio import features
from rasterio import warp
import rasterio.mask
from rasterio.enums import Resampling
from rasterio.merge import merge

import numpy as np
from PIL import Image

import matplotlib.pyplot as plt

from shapely.geometry import Point

from tqdm import tqdm

import os
import sys
module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup
    
    
module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/usr/local/python/lib/python3.10/site-packages/osmnx/utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


## Config notebook

In [2]:
# Set raster spectral bands and analysis equation
band_name_dict = {'green':[False], #If GSD(resolution) of band is different, set True.
                  'nir':[False], #If GSD(resolution) of band is different, set True.
                  'eq':["(green-nir)/(green+nir)"]}

# Set analysis name
index_analysis = 'ndwi'

# Set directory to save analysed rasters
tmp_dir = f'../../../data/processed/tmp_{index_analysis}/'

# Set desired hex resolution
res = [12]

# Set frequency of search for rasters (MS = Month Start)
freq = 'MS'

# Set start and end date for search (Sentinel-2 2A has images since mids 2015)
start_date = '2016-01-01'
end_date = '2022-12-31'

# Save?
save = True # True

# Del rasters after processing.
del_data = False # True

# city can be substituted by "place". (presa_laboca)
city = 'presa_laboca'

#Set filter if necessary. Defaults to "{}". Example: No images with cloud cover avobe 10%: {"eo:cloud_cover": {"lt": 10}}
query = {"eo:cloud_cover": {"lt": 10}}

# Set satellite. Defaults to "sentinel-2-l2a"
satellite = 'sentinel-2-l2a'

## Download data

### Download data - area of interest

In [3]:
#Load data
presa_original = gpd.read_file("../../../data/external/temporal_todocker/presa_laboca.gpkg")

Area of interest treatment

In [4]:
#Filter for relevant data
columns_tokeep = ['Name','geometry']
presa = presa_original[columns_tokeep]

#Create buffer for dam geometry
polygon = presa.to_crs("EPSG:6372").buffer(500)
polygon = polygon.to_crs("EPSG:4326")
polygon = gpd.GeoDataFrame(geometry=polygon).dissolve().geometry

#Review result
print(polygon.shape)
polygon.head(2)

(1,)


0    POLYGON ((-100.15925 25.44864, -100.15948 25.4...
Name: geometry, dtype: geometry

### Data download - Download and proccess rasters

In [6]:
#download_raster_from_pc(gdf, index_analysis, city, freq, start_date, end_date, tmp_dir, band_name_dict, query={}, satellite="sentinel-2-l2a"):
df_len = aup.download_raster_from_pc(polygon, index_analysis, city, freq, start_date, end_date, tmp_dir, band_name_dict, query = query, satellite = satellite)

 13%|████████████████▎                                                                                                            | 11/84 [43:44<4:50:18, 238.61s/it]ERROR 1: TIFFFillTile:Read error at row 9728, col 9728, tile 86; got 0 bytes, expected 363035
ERROR 1: TIFFReadEncodedTile() failed.
ERROR 1: IReadBlock failed at X offset 20, Y offset 3: TIFFReadEncodedTile() failed.
 43%|████████████████████████████████████████████████████▋                                                                      | 36/84 [1:18:31<1:48:31, 135.65s/it]ERROR 1: TIFFFillTile:Read error at row 0, col 0, tile 221; got 0 bytes, expected 356072
ERROR 1: TIFFReadEncodedTile() failed.
ERROR 1: IReadBlock failed at X offset 1, Y offset 10: TIFFReadEncodedTile() failed.
 63%|█████████████████████████████████████████████████████████████████████████████▌                                             | 53/84 [2:02:59<1:23:31, 161.65s/it]ERROR 1: TIFFFillTile:Read error at row 6656, col 6656, tile 256; got 0 by

## Debug paso por paso

#download_raster_from_pc(gdf, index_analysis, city, freq, start_date, end_date, tmp_dir, band_name_dict, query={}, satellite="sentinel-2-l2a"):
gdf = poly.copy()

poly = gdf.to_crs("EPSG:6372").buffer(500)
poly = poly.to_crs("EPSG:4326")
poly = gpd.GeoDataFrame(geometry=poly).dissolve().geometry

# Extracts coordinates from polygon as DataFrame
coord_val = poly.bounds
# Gets coordinates for bounding box
n = coord_val.maxy.max()
s = coord_val.miny.min()
e = coord_val.maxx.max()
w = coord_val.minx.min()

# Sets the coordinates for the area of interest
area_of_interest = {
    "type": "Polygon",
    "coordinates": [
        [
            [e, s],
            [w, s],
            [w, n],
            [e, n],
            [e, s],
        ]
    ],
}

# create time of interest
time_of_interest = aup.create_time_of_interest(start_date, end_date, freq=freq)

# gathers items for time and area of interest
items = aup.gather_items(time_of_interest, area_of_interest, query=query, satellite=satellite)

date_list = aup.available_datasets(items, satellite)

# create dictionary from links
band_name_list = list(band_name_dict.keys())[:-1]
assets_hrefs = aup.link_dict(band_name_list, items, date_list)

# analyze available data according to raster properties
df_len, missing_months = aup.df_date_links(assets_hrefs, start_date, end_date, 
                                       band_name_list, freq)
aup.available_data_check(df_len, missing_months) # test for missing months
missing_months

# creates raster and analyzes percentage of missing data points
df_len, missing_months = aup.df_date_links(assets_hrefs, start_date, end_date, 
                                       band_name_list, freq)
pct_missing = round(missing_months/len(df_len),2)*100
pct_missing

# raster cropping with bounding box from earlier 
bounding_box = gpd.GeoDataFrame(geometry=poly).envelope
gdf_bb = gpd.GeoDataFrame(gpd.GeoSeries(bounding_box), columns=['geometry'])

# create GeoDataFrame to test nan values in raster
gdf_raster_test = gdf.to_crs("EPSG:6372").buffer(1)
gdf_raster_test = gdf_raster_test.to_crs("EPSG:4326")
gdf_raster_test = gpd.GeoDataFrame(geometry=gdf_raster_test).dissolve()

# download raster data by month
df_len = aup.create_raster_by_month(
    df_len, index_analysis, city, tmp_dir,
    band_name_dict,date_list, gdf_raster_test,
    gdf_bb, area_of_interest, satellite, query=query)

missing_months = len(df_len.loc[df_len.data_id==0])
missing_months

df_len.head(50)

df_len.loc[4,'data_id'] = 0
df_len.loc[15,'data_id'] = 0
df_len.loc[21,'data_id'] = 0
df_len.loc[45,'data_id'] = 0
df_len.loc[81,'data_id'] = 0
df_len

df_len = aup.raster_interpolation(df_len, city, tmp_dir, index_analysis)

missing_months = len(df_len.loc[df_len.data_id==0])
missing_months

## End of debug

### Data download - Create hexgrid from area of interest

In [16]:
hex_gdf = gpd.GeoDataFrame(polygon)
hex_gdf['res'] = res[0]

if len(res)>1:
#If there is more than one resolution
    for r in range(res[0]+1,res[-1]+1): #Skips res 8 because, originally, res 8 is already in original hex_gdf
        hex_tmp = aup.create_hexgrid(polygon, r)
        hex_tmp.rename(columns={f'hex_id_{r}':'hex_id'}, inplace=True)
        hex_tmp['res'] = r
        hex_gdf = pd.concat([hex_gdf, hex_tmp],ignore_index = True, axis = 0)
        del hex_tmp

#If there is only one resolution
else:
    hex_gdf = aup.create_hexgrid(hex_gdf, res[0])
    hex_gdf.rename(columns={f'hex_id_{res[0]}':'hex_id'}, inplace=True)
    hex_gdf['res'] = res[0]

In [17]:
print(hex_gdf.shape)
hex_gdf.head(2)

(36345, 3)


,hex_id,geometry,res
0,8c48a2116541dff,"POLYGON ((-100.14410 25.44557, -100.14400 25.4...",12
1,8c48a2c496835ff,"POLYGON ((-100.14527 25.43236, -100.14517 25.4...",12


## Data processing

### Data processing - Raster to hex

#df_len.loc[11,'data_id'] = 0

In [20]:
hex_gdf_i = hex_gdf_res.copy()
r = res[0]
hex_raster_analysis, df_raster_analysis = aup.raster_to_hex_analysis(hex_gdf_i, df_len, index_analysis,tmp_dir, city, r)

 14%|████████▎                                                 | 1/7 [02:31<15:09, 151.60s/it]

 29%|████████████████▌                                         | 2/7 [07:38<19:06, 229.26s/it]


RasterioIOError: ../../../data/processed/tmp_ndwi/presa_laboca_ndwi_4_2018.tif: No such file or directory

## Data save

In [ ]:
# local save
if save:
    hex_raster_analysis.to_file(tmp_dir+'local_save/'+f'{city}_{index_analysis}_HexRes{r}_v{i}.geojson')
    df_raster_analysis.to_csv(tmp_dir+'local_save/'+f'{city}_{index_analysis}_HexRes{r}_v{i}.csv')